In [3]:
import pandas as pd
import numpy as np
import os
# 获取当前工作目录路径
current_dir = os.getcwd()

# 加载数据
cf_file = = os.path.join(current_dir,"cf_report_DWAS_input.csv")
cf_phys_file = = os.path.join(current_dir,"cf_report_DWAS_physical_input.csv")
cf = pd.read_csv(cf_file)
cf_phys = pd.read_csv(cf_phys_file)

# 过滤掉理解度低于 3 的记录
cfu = cf[cf['understand'] > 2]
cfpu = cf_phys[cf_phys['understand'] > 2]

# 列出问题字段
qfields = ['q'+str(item+1) for item in list(range(21))] + ['dwa_id']
pqfields = ['q'+str(item+1) for item in [21, 22]] + ['dwa_id']

# 计算每个问题的方差，过滤掉方差为 0 的行
qs = cfu[qfields]
qs['variance'] = qs[list(qs)[:-1]].var(axis=1)
qs2 = qs[qs['variance'] != 0]

# 计算每个问题的中位数
def getCenter(questionDF, grouper, center='median'):
    from scipy import stats
    if center == 'mode':
        return questionDF.groupby(grouper).agg(lambda x: stats.mode(x)[0][0]).reset_index()
    elif center == 'mean':
        return questionDF.groupby(grouper).mean().reset_index()
    else:
        return questionDF.groupby(grouper).median().reset_index()

# 计算每个问题的中位数
centerQ = getCenter(qs2, 'dwa_id', center='median')
centerQp = getCenter(cfpu, 'dwa_id', center='median')

# 反转问题 22 和 23 的评分
centerQp[['q22', 'q23']] = 6.0 - centerQp[['q22', 'q23']]

# 计算数据强度 (qD)
centerQ['qD'] = centerQ[['q15', 'q16', 'q17', 'q18']].max(axis=1)

# 合并数据
centerQ = pd.merge(centerQ, centerQp, on='dwa_id')

# 输出合并后的数据
print(centerQ.head())


C:\Users\for30\AppData\Local\Temp\ipykernel_20212\3575437558.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qs['variance'] = qs[list(qs)[:-1]].var(axis=1)


              dwa_id   q1   q2   q3   q4   q5   q6   q7   q8   q9  ...  \
0  4.A.1.a.1.I01.D01  4.0  5.0  3.0  4.0  2.0  4.0  3.0  3.0  3.0  ...   
1  4.A.1.a.1.I01.D02  3.5  4.0  4.5  4.0  4.0  4.0  3.5  3.0  3.5  ...   
2  4.A.1.a.1.I01.D03  3.0  1.5  3.0  3.0  3.0  2.5  3.0  4.0  3.0  ...   
3  4.A.1.a.1.I01.D04  4.0  4.0  3.0  3.0  3.5  3.5  4.0  3.5  3.5  ...   
4  4.A.1.a.1.I02.D01  4.0  4.0  4.5  3.5  3.5  3.5  4.0  3.0  4.0  ...   

   variance   qD      _unit_id  q22  q23  understand  \
0  1.661905  5.0  1.494202e+09  3.0  3.0         4.0   
1  0.397619  5.0  1.494203e+09  3.0  2.0         5.0   
2  1.047619  4.0  1.494203e+09  3.0  2.0         5.0   
3  0.614286  4.0  1.494202e+09  2.0  2.0         5.0   
4  0.538095  4.0  1.494203e+09  3.5  4.0         4.5   

   i_have_experience_andor_familiarity_with_the_following_industries_gold  \
0                                                NaN                        
1                                                NaN            

C:\Users\for30\AppData\Local\Temp\ipykernel_20212\3575437558.py:31: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  return questionDF.groupby(grouper).median().reset_index()


In [4]:
import pandas as pd
import numpy as np

# 加载数据
cf_file = r"C:\Users\for30\Downloads\cf_report_DWAS_input.csv"
cf_phys_file = r"C:\Users\for30\Downloads\cf_report_DWAS_physical_input.csv"
cf = pd.read_csv(cf_file)
cf_phys = pd.read_csv(cf_phys_file)

# 过滤掉理解度低于 3 的记录
cfu = cf[cf['understand'] > 2]
cfpu = cf_phys[cf_phys['understand'] > 2]

# 列出问题字段
qfields = ['q'+str(item+1) for item in list(range(21))] + ['dwa_id']
pqfields = ['q'+str(item+1) for item in [21, 22]] + ['dwa_id']

# 计算每个问题的方差，过滤掉方差为 0 的行
qs = cfu[qfields]
qs['variance'] = qs[list(qs)[:-1]].var(axis=1)
qs2 = qs[qs['variance'] != 0]

# 计算每个问题的中位数
def getCenter(questionDF, grouper, center='median'):
    from scipy import stats
    if center == 'mode':
        return questionDF.groupby(grouper).agg(lambda x: stats.mode(x)[0][0]).reset_index()
    elif center == 'mean':
        return questionDF.groupby(grouper).mean().reset_index()
    else:
        # 只对数值型列计算中位数
        return questionDF.groupby(grouper).median(numeric_only=True).reset_index()

# 计算每个问题的中位数
centerQ = getCenter(qs2, 'dwa_id', center='median')
centerQp = getCenter(cfpu, 'dwa_id', center='median')

# 反转问题 22 和 23 的评分
centerQp[['q22', 'q23']] = 6.0 - centerQp[['q22', 'q23']]  # 转换评分，6 - 评分

# 计算数据强度 (qD)
centerQ['qD'] = centerQ[['q15', 'q16', 'q17', 'q18']].max(axis=1)  # 计算问题15-18的最大值作为数据强度

# 合并数据
centerQ = pd.merge(centerQ, centerQp, on='dwa_id')

# 输出合并后的数据
print(centerQ.head())


              dwa_id   q1   q2   q3   q4   q5   q6   q7   q8   q9  ...  \
0  4.A.1.a.1.I01.D01  4.0  5.0  3.0  4.0  2.0  4.0  3.0  3.0  3.0  ...   
1  4.A.1.a.1.I01.D02  3.5  4.0  4.5  4.0  4.0  4.0  3.5  3.0  3.5  ...   
2  4.A.1.a.1.I01.D03  3.0  1.5  3.0  3.0  3.0  2.5  3.0  4.0  3.0  ...   
3  4.A.1.a.1.I01.D04  4.0  4.0  3.0  3.0  3.5  3.5  4.0  3.5  3.5  ...   
4  4.A.1.a.1.I02.D01  4.0  4.0  4.5  3.5  3.5  3.5  4.0  3.0  4.0  ...   

   variance   qD      _unit_id  q22  q23  understand  \
0  1.661905  5.0  1.494202e+09  3.0  3.0         4.0   
1  0.397619  5.0  1.494203e+09  3.0  2.0         5.0   
2  1.047619  4.0  1.494203e+09  3.0  2.0         5.0   
3  0.614286  4.0  1.494202e+09  2.0  2.0         5.0   
4  0.538095  4.0  1.494203e+09  3.5  4.0         4.5   

   i_have_experience_andor_familiarity_with_the_following_industries_gold  \
0                                                NaN                        
1                                                NaN            

C:\Users\for30\AppData\Local\Temp\ipykernel_20212\3926502625.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qs['variance'] = qs[list(qs)[:-1]].var(axis=1)


In [3]:
import pandas as pd
import numpy as np
import os
# 获取当前工作目录路径
current_dir = os.getcwd()

# 加载数据
cf_file = os.path.join(current_dir,"cf_report_DWAS_input.csv")
cf_phys_file = os.path.join(current_dir,"cf_report_DWAS_physical_input.csv")
cf = pd.read_csv(cf_file)
cf_phys = pd.read_csv(cf_phys_file)

# 过滤掉理解度低于 3 的记录
cfu = cf[cf['understand'] > 2]
cfpu = cf_phys[cf_phys['understand'] > 2]

# 列出问题字段
qfields = ['q'+str(item+1) for item in list(range(21))] + ['dwa_id']
pqfields = ['q'+str(item+1) for item in [21,22]] + ['dwa_id']
qs = cfu[qfields] # only the questions and task id
pqs = cfpu[pqfields]
qs['variance']=qs[list(qs)[:-1]].var(axis=1) #calculate row-wise variance 

# 计算每个问题的方差，过滤掉方差为 0 的行
qs = cfu[qfields]
pqs = cfpu[pqfields]
qs.loc[:, 'variance'] = qs[list(qs)[:-1]].var(axis=1)  # 使用 .loc 来显式赋值
qs2 = qs[qs['variance'] != 0]  


# 计算每个问题的中位数
def getCenter(questionDF, grouper, center='median'):
    from scipy import stats
    if center == 'mode':
        return questionDF.groupby(grouper).agg(lambda x: stats.mode(x)[0][0]).reset_index()
    elif center == 'mean':
        return questionDF.groupby(grouper).mean().reset_index()
    else:
        # 只对数值型列计算中位数
        return questionDF.groupby(grouper).median(numeric_only=True).reset_index()

# 计算每个问题的中位数
centerQ = getCenter(qs2, 'dwa_id', center='median')
centerQp = getCenter(cfpu, 'dwa_id', center='median')

# 反转问题 22 和 23 的评分
centerQp[['q22', 'q23']] = 6.0 - centerQp[['q22', 'q23']]  # 转换评分，6 - 评分

# 计算数据强度 (qD)
centerQ['qD'] = centerQ[['q15', 'q16', 'q17', 'q18']].max(axis=1)  # 计算问题15-18的最大值作为数据强度
qfields.insert(-1,'qD')
# 合并数据
centerQ = pd.merge(centerQ, centerQp, on='dwa_id')
qfields = qfields+pqfields[0:2]

# 输出合并后的数据
print(centerQ.head())

C:\Users\for30\AppData\Local\Temp\ipykernel_23692\950644044.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qs['variance']=qs[list(qs)[:-1]].var(axis=1) #calculate row-wise variance
C:\Users\for30\AppData\Local\Temp\ipykernel_23692\950644044.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qs.loc[:, 'variance'] = qs[list(qs)[:-1]].var(axis=1)  # 使用 .loc 来显式赋值


              dwa_id   q1   q2   q3   q4   q5   q6   q7   q8   q9  ...  \
0  4.A.1.a.1.I01.D01  4.0  5.0  3.0  4.0  2.0  4.0  3.0  3.0  3.0  ...   
1  4.A.1.a.1.I01.D02  3.5  4.0  4.5  4.0  4.0  4.0  3.5  3.0  3.5  ...   
2  4.A.1.a.1.I01.D03  3.0  1.5  3.0  3.0  3.0  2.5  3.0  4.0  3.0  ...   
3  4.A.1.a.1.I01.D04  4.0  4.0  3.0  3.0  3.5  3.5  4.0  3.5  3.5  ...   
4  4.A.1.a.1.I02.D01  4.0  4.0  4.5  3.5  3.5  3.5  4.0  3.0  4.0  ...   

   variance   qD      _unit_id  q22  q23  understand  \
0  1.661905  5.0  1.494202e+09  3.0  3.0         4.0   
1  0.397619  5.0  1.494203e+09  3.0  2.0         5.0   
2  1.047619  4.0  1.494203e+09  3.0  2.0         5.0   
3  0.614286  4.0  1.494202e+09  2.0  2.0         5.0   
4  0.538095  4.0  1.494203e+09  3.5  4.0         4.5   

   i_have_experience_andor_familiarity_with_the_following_industries_gold  \
0                                                NaN                        
1                                                NaN            

In [4]:
# 加载任务数据
task_file = os.path.join(current_dir,"Tasks to DWAs.xlsx")
tasks = pd.read_excel(task_file)

# 合并 DWA 分数到任务数据中
tasksDWAs = tasks.merge(centerQ, left_on=['DWA ID'], right_on=['dwa_id'], how='left')

# 计算任务级别的分数
def getCenter(questionDF, grouper, center='median'):
    from scipy import stats
    if center == 'mode':
        return questionDF.groupby(grouper).agg(lambda x: stats.mode(x)[0][0]).reset_index()
    elif center == 'mean':
        return questionDF.groupby(grouper).mean(numeric_only=True).reset_index()  # 显式指定 numeric_only=True
    else:
        return questionDF.groupby(grouper).median(numeric_only=True).reset_index()

# 计算任务级别的均值
taskScores = getCenter(tasksDWAs[['Task ID'] + qfields], 'Task ID', center='mean')

# 合并任务数据和分数
onlyTasks = tasks[['O*NET-SOC Code', 'Title', 'Task ID', 'Task']].drop_duplicates()
taskScores2 = taskScores.merge(onlyTasks, on='Task ID')

# 保存任务分数到 CSV 文件
taskScores2.to_csv('Tasks_Scores.csv', index=False)

In [6]:
# 加载工作数据（任务评级和任务说明）
jobTaskRatingFile = os.path.join(current_dir,"Task Ratings.xlsx")
jobs = pd.read_excel(jobTaskRatingFile)

# 获取任务的重要性值
jobTask = jobs[jobs['Scale ID'] == 'IM'][['O*NET-SOC Code', 'Title', 'Task ID', 'Task', 'Data Value']]
jobTaskScores = pd.merge(jobTask, taskScores2, on=['Task ID', 'O*NET-SOC Code', 'Title', 'Task'])

# 标准化数据值
jobTaskScores['Data Value'] = jobTaskScores['Data Value'] / 5.0  # scale to 0-1
jobTaskScores['weight'] = jobTaskScores['Data Value'] / jobTaskScores.groupby('O*NET-SOC Code')['Data Value'].transform('sum')
JTS = jobTaskScores.copy()  # prevent overwriting

# 计算职位的加权平均分数
for q in list(JTS)[5:-1]:
    JTS['w' + q] = JTS[q] * JTS['weight']
JTSfields = [item for item in list(JTS) if item[0:2] == 'wq']
print(JTSfields)
wJTS = JTS.groupby(['O*NET-SOC Code', 'Title'])[JTSfields].sum().reset_index()

jobScores = wJTS  # 只保留计算出的加权分数，而不再合并额外的职位数据

# 保存职位分数到 CSV 文件
jobScores.to_csv('Job_Scores.csv', index=False)  # save jobScores for later

# 方差计算
JTSV = jobTaskScores.copy()  # separate copy for the variances
vJTS = JTSV.merge(wJTS, on=['O*NET-SOC Code', 'Title'], how='left')
for q in [k for k in list(vJTS) if k[0] == 'q']:  # for all the original question values
    vJTS['dV' + q] = vJTS['weight'] * (vJTS[q] - vJTS['w' + q]) ** 2  # dV is for sq. deviation value

vJTSfields = [item for item in list(vJTS) if item[0:2] == 'dV']
wvJTS = vJTS.groupby(['O*NET-SOC Code', 'Title'])[vJTSfields].sum().reset_index()

jobVarianceScores = wvJTS  # 只保留计算的方差分数

# 保存方差分数到 CSV 文件
jobVarianceScores.to_csv('JobVariance_Scores.csv', index=False)  # save variance scores for later

# 合并加权分数和方差分数（不再包括职位数据）
allscores = pd.merge(jobScores, wvJTS, on=['O*NET-SOC Code', 'Title'])

# 保存加权分数和方差分数合并后的数据
allscores.to_csv('JobScores_mean_variance.csv', index=False)  # save allscores

# 分类问题为 SML 或 Measurability
SML = ['q3', 'q5', 'q6', 'q7', 'q8', 'q9', 'q10', 'q11', 'q12', 'q13', 'q14', 'q19', 'q20', 'q21', 'q22', 'q23']  # SML without measurement/data fields
measurability = ['q1', 'q2', 'q4', 'qD']  # Measurement and data fields - maximum is applied twice to increase "weight"

# 计算每个问题的均值和方差
allscores['mSML'] = (1 / float(len(SML + measurability))) * allscores[['w' + item for item in SML + measurability]].sum(axis=1)
allscores['vmSML'] = (1 / float(len(SML + measurability))) * allscores[['dV' + item for item in SML + measurability]].sum(axis=1)
allscores['sdmSML'] = np.sqrt(allscores['vmSML'])

# 保存 SML 计算结果
allscores.to_csv('allscores_SML.csv', index=False)

['wq1', 'wq2', 'wq3', 'wq4', 'wq5', 'wq6', 'wq7', 'wq8', 'wq9', 'wq10', 'wq11', 'wq12', 'wq13', 'wq14', 'wq15', 'wq16', 'wq17', 'wq18', 'wq19', 'wq20', 'wq21', 'wqD', 'wq22', 'wq23']


In [21]:
# 读取文件
matched_result_jobs = pd.read_excel(os.path.join(current_dir, "matchedjobs最终处理结果.xlsx"))
allscores_SML = pd.read_csv(os.path.join(current_dir, "allscores_SML.csv"))

# 去除空格字符（确保合并时不出错）
matched_result_jobs['ONET_code'] = matched_result_jobs['ONET_code'].str.strip()
allscores_SML['O*NET-SOC Code'] = allscores_SML['O*NET-SOC Code'].str.strip()

# 确保 ONet_code 和 O*NET-SOC Code 都是字符串类型
matched_result_jobs['ONET_code'] = matched_result_jobs['ONET_code'].astype(str)
allscores_SML['O*NET-SOC Code'] = allscores_SML['O*NET-SOC Code'].astype(str)

# 第一步：将 matched_result_jobs 与 allscores_SML 合并，获取 mSML、vmSML 和 sdmSML
merged = pd.merge(matched_result_jobs, allscores_SML[['O*NET-SOC Code', 'mSML', 'vmSML', 'sdmSML']], 
                  left_on='ONET_code', right_on='O*NET-SOC Code', how='left')

# 第二步：查看哪些行没有找到匹配项
unmatched = merged[merged['mSML'].isna()]  # 找到没有匹配的行
print("未匹配的数据行：")
print(unmatched[['ONET_code', 'O*NET-SOC Code']])  # 打印未匹配的 ONET_code 和 O*NET-SOC Code

# 将未匹配的结果保存为文件，以供进一步分析
unmatched.to_csv("unmatched_jobs.csv", index=False, encoding='utf-8-sig')

# 第三步：将结果与 result_df 合并，获取中文职业名称
result_file = os.path.join(current_dir, "result中英版 - 工作表1.csv")
result_df = pd.read_csv(result_file, encoding='gbk')

merged = pd.merge(merged, result_df[['Code', 'ChineseTitle']], 
                  left_on='职业代码', right_on='Code', how='left')

# 第四步：创建最终的 DataFrame，选取所需的列
final_result = merged[['职业代码', 'ChineseTitle','O*NET-SOC Code', 'mSML']]

# 重命名列名
final_result.columns = ['职业代码', '职业名称', 'O*NET-SOC Code', 'SML']

# 保存结果
final_result.to_csv("final_result.csv", index=False, encoding='utf-8-sig')

# 打印最终结果的前几行
print(final_result.head())

未匹配的数据行：
       ONET_code O*NET-SOC Code
11    43-9199.00            NaN
15    11-2033.00            NaN
21    11-9199.00            NaN
22    13-1199.00            NaN
32    19-1029.00            NaN
...          ...            ...
4180  43-2099.00            NaN
4191  53-6032.00            NaN
4193  53-6032.00            NaN
4194  53-6032.00            NaN
4234  49-9069.00            NaN

[195 rows x 2 columns]


PermissionError: [Errno 13] Permission denied: 'unmatched_jobs.csv'

In [26]:
import pandas as pd
import os

# 读取文件
matched_result_jobs = pd.read_excel(os.path.join(current_dir, "matchedjobs最终处理结果.xlsx"))
allscores_SML = pd.read_csv(os.path.join(current_dir, "allscores_SML.csv"))

# 读取Occupational Data 
occupation_data_file = os.path.join(current_dir, "Occupation Data.csv")
occupation_data = pd.read_csv(occupation_data_file)

# 去除空格字符（确保合并时不出错）
matched_result_jobs['ONET_code'] = matched_result_jobs['ONET_code'].str.strip()
allscores_SML['O*NET-SOC Code'] = allscores_SML['O*NET-SOC Code'].str.strip()

# 确保 ONet_code 和 O*NET-SOC Code 都是字符串类型
matched_result_jobs['ONET_code'] = matched_result_jobs['ONET_code'].astype(str)
allscores_SML['O*NET-SOC Code'] = allscores_SML['O*NET-SOC Code'].astype(str)

# 第一步：将 matched_result_jobs 与 allscores_SML 合并，获取 mSML、vmSML 和 sdmSML
merged = pd.merge(matched_result_jobs, allscores_SML[['O*NET-SOC Code', 'mSML', 'vmSML', 'sdmSML']], 
                  left_on='ONET_code', right_on='O*NET-SOC Code', how='left')

# 查看哪些行没有找到匹配项
unmatched = merged[merged['mSML'].isna()]  # 找到没有匹配的行
print("未匹配的数据行：")
print(unmatched[['ONET_code', 'O*NET-SOC Code']])  # 打印未匹配的 ONET_code 和 O*NET-SOC Code

# 将未匹配的结果保存为文件，以供进一步分析
unmatched.to_csv("unmatched_jobs.csv", index=False, encoding='utf-8-sig')

# 计算 unmatched_jobs 中不相同的 ONET_code 数量
unique_unmatched_onet_codes = unmatched['ONET_code'].nunique()
print(f"未匹配的不同的 ONET_code 数量: {unique_unmatched_onet_codes}")

# 获取未匹配的 ONET_code 列
unmatched_onet_codes = unmatched['ONET_code'].unique()

# 从 Occupation Data 中删除这些 ONET_code 对应的行
occupation_data_filtered = occupation_data[~occupation_data['Code'].isin(unmatched_onet_codes)]

# 计算删除的行数
deleted_rows_count = len(occupation_data) - len(occupation_data_filtered)
print(f"删除的行数: {deleted_rows_count}")

# 保存过滤后的 Occupation Data
occupation_data_filtered.to_csv("filtered_occupation_data.csv", index=False, encoding='utf-8-sig')

未匹配的数据行：
       ONET_code O*NET-SOC Code
11    43-9199.00            NaN
15    11-2033.00            NaN
21    11-9199.00            NaN
22    13-1199.00            NaN
32    19-1029.00            NaN
...          ...            ...
4180  43-2099.00            NaN
4191  53-6032.00            NaN
4193  53-6032.00            NaN
4194  53-6032.00            NaN
4234  49-9069.00            NaN

[195 rows x 2 columns]
未匹配的不同的 ONET_code 数量: 55
删除的行数: 55
